# Genre Collaboration Network (2000-2023)

Network visualization from BigQuery co-occurrence data.

In [123]:
# Load packages
library(tidyverse)
library(igraph)
library(GGally)
library(network)
library(sna)
library(intergraph)
library(patchwork)

theme_set(theme_minimal())

## Load Data

In [124]:
# Load co-occurrence network
edges_all <- read_csv(
  '../../data/sql_query_out/QUERY 2_ Genre Co-Occurrence Network (All Time).csv',
  show_col_types = FALSE
) %>%
  rename(from = genre_1, to = genre_2, weight = co_occurrence_count) %>%
  filter(from != to, weight >= 2)

cat(sprintf('%d genre connections loaded\n', nrow(edges_all)))

edges_all %>% arrange(desc(weight)) %>% head(10)

8597 genre connections loaded


from,to,weight
<chr>,<chr>,<dbl>
hip hop,rap,287
pop rap,rap,273
dance pop,pop,268
mellow gold,soft rock,243
album rock,rock,239
pop rap,southern hip hop,237
r&b,urban contemporary,224
hip hop,pop rap,218
rap,southern hip hop,213


In [125]:
# Load genre mapping
genre_mapping <- read_csv(
  '../../data/sql_query_out/QUERY 5_ Genre to Main Genre Mapping (For Node Coloring).csv',
  show_col_types = FALSE
) %>%
  mutate(primary_main_genre = tolower(trimws(primary_main_genre)))

cat(sprintf('%d subgenres mapped\n', nrow(genre_mapping)))

genre_mapping %>% count(primary_main_genre, sort = TRUE) %>% head(10)

1562 subgenres mapped


primary_main_genre,n
<chr>,<int>
alternative metal,70
alternative rock,44
alternative dance,43
album rock,36
dance pop,29
alternative country,22
latin,22
anthem worship,19
big room,18


## Build Network

In [126]:
# Build node list with macro genre mapping
all_genres <- unique(c(edges_all$from, edges_all$to))

# Load complete mapping (regex + manual assignments)
macro_mapping <- read_csv('../../data/cleaned/genre_network_mapping.csv', show_col_types = FALSE)

nodes <- tibble(name = all_genres) %>%
  left_join(genre_mapping %>% select(sub_genre, artist_count),
            by = c('name' = 'sub_genre')) %>%
  left_join(macro_mapping, by = c('name' = 'micro_genre')) %>%
  mutate(
    artist_count = ifelse(is.na(artist_count), 1, artist_count),
    macro_genre = ifelse(is.na(macro_genre), 'OTHER', macro_genre)
  )

cat(sprintf('%d nodes, %d edges\n', nrow(nodes), nrow(edges_all)))
cat('\nMacro genre distribution:\n')
nodes %>% count(macro_genre, sort = TRUE) %>% print(n = 20)

957 nodes, 8597 edges

Macro genre distribution:
# A tibble: 16 x 2
   macro_genre     n
   <chr>       <int>
 1 ROCK          239
 2 POP           129
 3 ELECTRONIC     93
 4 METAL          86
 5 HIP HOP        80
 6 R&B            62
 7 JAZZ           45
 8 LATIN          43
 9 OTHER          39
10 CLASSICAL      36
11 FOLK           33
12 COUNTRY        32
13 BLUES          17
14 REGGAE         11
15 NEW AGE         8
16 AVANT-GARDE     4


## Network Setup

In [127]:
# 16 macro genre colors
macro_genre_colors <- c(
  'POP' = '#f180a6ff',
  'ELECTRONIC' = '#d31f8eff',
  'R&B' = '#FF0800',
  'HIP HOP' = '#9D2A3A',
  'REGGAE' = '#FFB627',
  'LATIN' = '#E07B00',
  'BLUES' = '#1565C0',
  'JAZZ' = '#5C9CE6',
  'ROCK' = '#2832C2',
  'METAL' = '#1F456E',
  'FOLK' = '#6B8E4E',
  'COUNTRY' = '#7bac21ff',
  'CLASSICAL' = '#7B2D8E',
  'NEW AGE' = '#B57EDC',
  'AVANT-GARDE' = '#4A235A',
  'OTHER' = '#757575'
)

# Create igraph
g <- igraph::graph_from_data_frame(edges_all, directed = FALSE, vertices = nodes)

# Convert to network object
net <- intergraph::asNetwork(g)
net %v% "genre_name" <- nodes$name
net %v% "artist_count" <- nodes$artist_count
net %v% "macro_genre" <- nodes$macro_genre

# Layout
set.seed(42)
layout <- igraph::layout_with_fr(g, weights = 1/igraph::E(g)$weight, niter = 5000) * 1.5

## Hub Genres

In [128]:
# Calculate centrality
hub_metrics <- tibble(
  genre = nodes$name,
  main_genre = nodes$macro_genre,
  artists = nodes$artist_count,
  degree = igraph::degree(g),
  strength = igraph::strength(g),
  betweenness = igraph::betweenness(g, weights = 1/igraph::E(g)$weight)
) %>%
  arrange(desc(strength))

head(hub_metrics, 20)

# Bar chart
p_hubs <- ggplot(head(hub_metrics, 25), aes(x = reorder(genre, strength), y = strength, fill = main_genre)) +
  geom_col() +
  coord_flip() +
  labs(title = 'Top 25 Hub Genres', x = NULL, y = 'Collaboration Strength', fill = 'Genre') +
  scale_fill_manual(values = macro_genre_colors) +
  theme_minimal()

ggsave('../../outputs/genre_network/genre_hubs.png', p_hubs, width = 12, height = 10, dpi = 300, bg = 'white')

genre,main_genre,artists,degree,strength,betweenness
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
rock,ROCK,234,172,4062,162032.25
folk rock,ROCK,89,105,2846,45627.00
singer-songwriter,FOLK,54,140,2740,14171.00
modern rock,ROCK,62,153,2723,59555.43
mellow gold,POP,145,96,2591,33109.00
indie rock,ROCK,69,128,2517,16454.51
folk-pop,POP,46,145,2427,28311.01
classic rock,ROCK,172,95,2390,7482.00
hip hop,HIP HOP,94,79,2330,31245.00


In [129]:
# Full network visualization - FR layout with center scaling and collision detection
library(ggraph)

set.seed(42)

# Select top 15% of nodes by artist count for labels
label_threshold <- quantile(nodes$artist_count, 0.75)
nodes_for_plot <- nodes %>%
  mutate(
    # Wrap text - replace spaces with newlines for multi-word genres
    label = ifelse(artist_count >= label_threshold, 
                   str_replace_all(name, " ", "\n"), 
                   NA_character_)
  )

# Create fresh graph
g_full <- igraph::graph_from_data_frame(edges_all, directed = FALSE, vertices = nodes_for_plot)

# Calculate layout first, then scale it to spread nodes
layout_fr <- create_layout(g_full, layout = 'fr', weights = 1/E(g_full)$weight, niter = 10000)

# Scale coordinates outward from center
center_x <- mean(layout_fr$x)
center_y <- mean(layout_fr$y)
layout_fr$x <- center_x + (layout_fr$x - center_x) * 3.5
layout_fr$y <- center_y + (layout_fr$y - center_y) * 3.5

# Collision detection - push overlapping nodes apart
# Node radius proportional to artist_count
min_size <- 2
max_size <- 18
layout_fr$radius <- min_size + (max_size - min_size) * 
  (layout_fr$artist_count - min(layout_fr$artist_count)) / 
  (max(layout_fr$artist_count) - min(layout_fr$artist_count))

# Iterative collision resolution
for (iter in 1:50) {
  moved <- FALSE
  for (i in 1:(nrow(layout_fr) - 1)) {
    for (j in (i + 1):nrow(layout_fr)) {
      dx <- layout_fr$x[j] - layout_fr$x[i]
      dy <- layout_fr$y[j] - layout_fr$y[i]
      dist <- sqrt(dx^2 + dy^2)
      min_dist <- (layout_fr$radius[i] + layout_fr$radius[j]) * 0.35  # scale factor for plot units
      
      if (dist < min_dist && dist > 0) {
        # Push nodes apart
        overlap <- min_dist - dist
        pushx <- (dx / dist) * overlap * 0.5
        pushy <- (dy / dist) * overlap * 0.5
        
        layout_fr$x[i] <- layout_fr$x[i] - pushx
        layout_fr$y[i] <- layout_fr$y[i] - pushy
        layout_fr$x[j] <- layout_fr$x[j] + pushx
        layout_fr$y[j] <- layout_fr$y[j] + pushy
        moved <- TRUE
      }
    }
  }
  if (!moved) break
}
cat(sprintf("Collision resolution: %d iterations\n", iter))

p_full <- ggraph(layout_fr) +
  geom_edge_link(alpha = 0.05, color = "gray60") +
  geom_node_point(aes(color = macro_genre, size = artist_count), alpha = 0.8) +
  geom_node_text(aes(label = label), size = 1.2, color = "#ffffffff", fontface = "bold", lineheight = 0.8) +
  scale_color_manual(values = macro_genre_colors, name = "Genre") +
  scale_size_continuous(range = c(2, 18), name = "Artists") +
  labs(title = "Billboard Genre Network (2000-2023)",
       subtitle = sprintf("%d nodes, %d edges | Top 15%% labeled", vcount(g_full), ecount(g_full))) +
  theme_void() +
  theme(
    plot.title = element_text(hjust = 0.5, size = 20, face = "bold"),
    plot.subtitle = element_text(hjust = 0.5, size = 12, color = "gray40"),
    legend.position = "right",
    plot.margin = margin(20, 20, 20, 20)
  ) +
  coord_fixed(ratio = 1)

ggsave('../../outputs/genre_network/genre_network_full.png', p_full, 
       width = 28, height = 24, dpi = 300, bg = 'white')

cat(sprintf("Saved genre_network_full.png (%d nodes, %d edges)\n", vcount(g_full), ecount(g_full)))

Collision resolution: 50 iterations


Warning message:
"Removed 709 rows containing missing values or values outside the scale range
(`geom_text()`)."


Saved genre_network_full.png (957 nodes, 8597 edges)


## Yearly Networks

In [130]:
# Create network for a given year - FR layout with "pop" fixed at center
library(ggraph)

create_year_network_circular <- function(year_val) {
  file_path <- sprintf('../../data/sql_query_out/QUERY 4_ Export Individual Year Networks (Example for %d).csv', year_val)
  if (!file.exists(file_path)) return(NULL)
  
  edges_yr <- read_csv(file_path, show_col_types = FALSE) %>%
    rename(from = genre_1, to = genre_2, weight = co_occurrence_count) %>%
    filter(from != to, weight >= 2)
  
  if (nrow(edges_yr) == 0) return(NULL)
  
  genres_yr <- unique(c(edges_yr$from, edges_yr$to))
  nodes_yr <- nodes %>% filter(name %in% genres_yr)
  
  # Ensure "pop" is included
  if (!"pop" %in% nodes_yr$name) {
    pop_node <- nodes %>% filter(name == "pop")
    if (nrow(pop_node) > 0) {
      nodes_yr <- bind_rows(nodes_yr, pop_node)
    }
  }
  
  # Create graph with vertex attributes
  g_yr <- igraph::graph_from_data_frame(edges_yr, directed = FALSE, vertices = nodes_yr)
  
  # Set vertex attributes explicitly
  V(g_yr)$artist_count <- nodes_yr$artist_count[match(V(g_yr)$name, nodes_yr$name)]
  V(g_yr)$macro_genre <- nodes_yr$macro_genre[match(V(g_yr)$name, nodes_yr$name)]
  
  # Calculate FR layout
  set.seed(42)
  layout_mat <- igraph::layout_with_fr(g_yr, weights = 1/E(g_yr)$weight, niter = 5000)
  
  # Find "pop" and move it to center, shift all other nodes accordingly
  node_names <- V(g_yr)$name
  pop_idx <- which(node_names == "pop")
  
  if (length(pop_idx) > 0) {
    # Get pop's current position
    pop_x <- layout_mat[pop_idx, 1]
    pop_y <- layout_mat[pop_idx, 2]
    
    # Shift all nodes so pop is at origin
    layout_mat[, 1] <- layout_mat[, 1] - pop_x
    layout_mat[, 2] <- layout_mat[, 2] - pop_y
  }
  
  # Scale layout for better spread
  layout_mat <- layout_mat * 2
  
  # Create ggraph layout
  layout_df <- create_layout(g_yr, layout = 'manual', x = layout_mat[,1], y = layout_mat[,2])
  
  # Select top nodes for labels (by artist_count)
  label_threshold <- quantile(layout_df$artist_count, 0.7, na.rm = TRUE)
  layout_df$label <- ifelse(
    layout_df$artist_count >= label_threshold | layout_df$name == "pop",
    str_replace_all(layout_df$name, " ", "\n"),
    NA_character_
  )
  
  p <- ggraph(layout_df) +
    geom_edge_link(alpha = 0.1, color = "gray50") +
    geom_node_point(aes(color = macro_genre, size = artist_count), alpha = 0.85) +
    geom_node_text(aes(label = label), size = 2, color = "#333333", fontface = "bold", lineheight = 0.8) +
    scale_color_manual(values = macro_genre_colors, name = "Genre") +
    scale_size_continuous(range = c(2, 12), name = "Artists") +
    labs(title = as.character(year_val)) +
    theme_void() +
    theme(
      plot.title = element_text(hjust = 0.5, size = 18, face = "bold"),
      legend.position = "none"
    ) +
    coord_fixed(ratio = 1)
  
  return(p)
}

# Key years
key_years <- c(2000, 2005, 2010, 2015, 2020, 2023)
yearly_plots <- map(key_years, create_year_network_circular)
yearly_plots <- yearly_plots[!sapply(yearly_plots, is.null)]

if (length(yearly_plots) > 0) {
  combined <- wrap_plots(yearly_plots, ncol = 3) +
    plot_annotation(title = 'Genre Network Evolution (2000-2023)',
                    subtitle = 'Force-directed layout with POP anchored at center',
                    theme = theme(
                      plot.title = element_text(hjust = 0.5, size = 20, face = "bold"),
                      plot.subtitle = element_text(hjust = 0.5, size = 14, color = "gray40")
                    ))
  
  ggsave('../../outputs/genre_network/genre_network_evolution_key_years.png', combined,
         width = 28, height = 19, dpi = 300, bg = 'white')
  cat("Saved genre_network_evolution_key_years.png\n")
}

Warning message:
"Removed 31 rows containing missing values or values outside the scale range
(`geom_text()`)."
Warning message:
"Removed 31 rows containing missing values or values outside the scale range
(`geom_text()`)."
Warning message:
"Removed 25 rows containing missing values or values outside the scale range
(`geom_text()`)."
Warning message:
"Removed 27 rows containing missing values or values outside the scale range
(`geom_text()`)."
Warning message:
"Removed 17 rows containing missing values or values outside the scale range
(`geom_text()`)."
Warning message:
"Removed 12 rows containing missing values or values outside the scale range
(`geom_text()`)."


Saved genre_network_evolution_key_years.png


## Export Data

In [131]:
# Export all years for animation - circular layout with "pop" at center
dir.create('../../outputs/genre_network/genre_snapshots_yearly', showWarnings = FALSE, recursive = TRUE)

for (yr in 2000:2023) {
  p <- create_year_network_circular(yr)
  if (!is.null(p)) {
    ggsave(sprintf('../../outputs/genre_network/genre_snapshots_yearly/network_%d.png', yr),
           p, width = 12, height = 12, dpi = 150, bg = 'white')
    cat(sprintf('%d ', yr))
  }
}
cat('\nDone - Circular layout snapshots exported\n')

Warning message:
"Removed 31 rows containing missing values or values outside the scale range
(`geom_text()`)."


2000 

Warning message:
"Removed 35 rows containing missing values or values outside the scale range
(`geom_text()`)."


2001 

Warning message:
"Removed 35 rows containing missing values or values outside the scale range
(`geom_text()`)."


...

2021 

Warning message:
"Removed 13 rows containing missing values or values outside the scale range
(`geom_text()`)."


2022 

Warning message:
"Removed 12 rows containing missing values or values outside the scale range
(`geom_text()`)."


2023 
Done - Circular layout snapshots exported


## Temporal Edges Export for Gephi

Create a single edges file with a year column so edges can be toggled on/off by year in Gephi.

In [132]:
# Create temporal edges file with year column for Gephi dynamic filtering
# Each unique edge pair (from, to) will have 24 rows - one per year (2000-2023)
# Weight = 0 if no connection that year, otherwise the co-occurrence count

# Load all yearly data files and combine
years <- 2000:2023

yearly_edges_list <- map(years, function(yr) {
  file_path <- sprintf('../../data/sql_query_out/QUERY 4_ Export Individual Year Networks (Example for %d).csv', yr)
  if (!file.exists(file_path)) return(NULL)
  
  read_csv(file_path, show_col_types = FALSE) %>%
    rename(from = genre_1, to = genre_2, weight = co_occurrence_count) %>%
    filter(from != to) %>%  # Remove self-loops
    mutate(year = yr)
})

yearly_edges <- bind_rows(yearly_edges_list)
cat(sprintf('Loaded %d yearly edge records\n', nrow(yearly_edges)))

# Get all unique edge pairs from the all-time network (these are canonical pairs)
all_edge_pairs <- edges_all %>%
  select(from, to) %>%
  # Ensure consistent ordering (alphabetical) to avoid duplicates
  mutate(
    genre_a = pmin(from, to),
    genre_b = pmax(from, to)
  ) %>%
  select(genre_a, genre_b) %>%
  distinct()

cat(sprintf('%d unique edge pairs in all-time network\n', nrow(all_edge_pairs)))

# Create full grid: all edge pairs × all years
temporal_grid <- expand_grid(
  all_edge_pairs,
  year = years
)

cat(sprintf('Grid size: %d edge pairs × %d years = %d rows\n', 
            nrow(all_edge_pairs), length(years), nrow(temporal_grid)))

# Normalize yearly edges to same ordering for matching
yearly_edges_normalized <- yearly_edges %>%
  mutate(
    genre_a = pmin(from, to),
    genre_b = pmax(from, to)
  ) %>%
  group_by(genre_a, genre_b, year) %>%
  summarize(weight = sum(weight), .groups = 'drop')

# Join to get weights (0 if no connection that year)
temporal_edges <- temporal_grid %>%
  left_join(yearly_edges_normalized, by = c('genre_a', 'genre_b', 'year')) %>%
  mutate(weight = ifelse(is.na(weight), 0, weight)) %>%
  rename(from = genre_a, to = genre_b)

# Summary stats
cat('\nTemporal edges summary:\n')
cat(sprintf('  Total rows: %d\n', nrow(temporal_edges)))
cat(sprintf('  Non-zero weights: %d (%.1f%%)\n', 
            sum(temporal_edges$weight > 0),
            100 * sum(temporal_edges$weight > 0) / nrow(temporal_edges)))

# Show weight distribution by year
temporal_edges %>%
  group_by(year) %>%
  summarize(
    edges_active = sum(weight > 0),
    total_weight = sum(weight),
    .groups = 'drop'
  ) %>%
  print(n = 24)

Loaded 8870 yearly edge records
8597 unique edge pairs in all-time network
Grid size: 8597 edge pairs <U+00D7> 24 years = 206328 rows

Temporal edges summary:
  Total rows: 206328
  Non-zero weights: 8035 (3.9%)
# A tibble: 24 x 3
    year edges_active total_weight
   <int>        <int>        <dbl>
 1  2000          348         1010
 2  2001          428         1258
 3  2002          417         1162
 4  2003          466         1511
 5  2004          451         1453
 6  2005          455         1598
 7  2006          383         1364
 8  2007          378         1331
 9  2008          347         1036
10  2009          331          942
11  2010          347          968
12  2011          375         1081
13  2012          305          736
14  2013          370          917
15  2014          432         1158
16  2015          454          969
17  2016          325          858
18  2017          257          756
19  2018          231          692
20  2019          174          415

In [133]:
# Export temporal edges - same format as genre_network_edges_gephi.csv but with year column
# Format: from,to,weight,year

temporal_edges_export <- temporal_edges %>%
  # Keep same column names as original: from, to, weight
  # Just add year column
  select(from, to, weight, year) %>%
  # Sort by edge pair then year
  arrange(from, to, year)

# Preview - show dance pop -> pop across all years
cat('Sample of temporal edges (dance pop <-> pop):\n')
temporal_edges_export %>% 
  filter(from == 'dance pop', to == 'pop') %>%
  print(n = 24)

# Export
write_csv(temporal_edges_export, '../../outputs/genre_network/genre_network_edges_temporal_gephi.csv')

cat(sprintf('\nExported: genre_network_edges_temporal_gephi.csv\n'))
cat(sprintf('  %d rows (%d edge pairs × %d years)\n', 
            nrow(temporal_edges_export), 
            n_distinct(paste(temporal_edges_export$from, temporal_edges_export$to)),
            length(unique(temporal_edges_export$year))))
cat(sprintf('  Columns: from, to, weight, year\n'))
cat(sprintf('  (Same format as genre_network_edges_gephi.csv + year column)\n'))

Sample of temporal edges (dance pop <-> pop):
# A tibble: 24 x 4
   from      to    weight  year
   <chr>     <chr>  <dbl> <int>
 1 dance pop pop       13  2000
 2 dance pop pop       19  2001
 3 dance pop pop       21  2002
 4 dance pop pop       17  2003
 5 dance pop pop       14  2004
 6 dance pop pop       27  2005
 7 dance pop pop       25  2006
 8 dance pop pop       34  2007
 9 dance pop pop       31  2008
10 dance pop pop       31  2009
11 dance pop pop       41  2010
12 dance pop pop       45  2011
13 dance pop pop       33  2012
14 dance pop pop       41  2013
15 dance pop pop       41  2014
16 dance pop pop       38  2015
17 dance pop pop       35  2016
18 dance pop pop       29  2017
19 dance pop pop       26  2018
20 dance pop pop       18  2019
21 dance pop pop       16  2020
22 dance pop pop       12  2021
23 dance pop pop       11  2022
24 dance pop pop       17  2023

Exported: genre_network_edges_temporal_gephi.csv
  206328 rows (8595 edge pairs <U+00D7> 24 years)
  C

In [134]:
# Export original edges CSV (all-time, no year column)
# Format: from,to,weight

edges_export <- edges_all %>%
  select(from, to, weight) %>%
  arrange(desc(weight))

write_csv(edges_export, '../../outputs/genre_network/genre_network_edges_gephi.csv')

cat(sprintf('Exported: genre_network_edges_gephi.csv\n'))
cat(sprintf('  %d edges\n', nrow(edges_export)))
cat(sprintf('  Columns: from, to, weight\n\n'))

# Export nodes CSV
# Format: name,primary_main_genre,artist_count,macro_genre

nodes_export <- nodes %>%
  left_join(genre_mapping %>% select(sub_genre, primary_main_genre), 
            by = c('name' = 'sub_genre')) %>%
  select(name, primary_main_genre, artist_count, macro_genre)

write_csv(nodes_export, '../../outputs/genre_network/genre_network_nodes_gephi.csv')

cat(sprintf('Exported: genre_network_nodes_gephi.csv\n'))
cat(sprintf('  %d nodes\n', nrow(nodes_export)))
cat(sprintf('  Columns: name, primary_main_genre, artist_count, macro_genre\n'))

Exported: genre_network_edges_gephi.csv
  8597 edges
  Columns: from, to, weight

Exported: genre_network_nodes_gephi.csv
  957 nodes
  Columns: name, primary_main_genre, artist_count, macro_genre
